In [40]:
from pygeoinf.interval.function_providers import NormalModesProvider, BumpFunctionProvider
from pygeoinf.interval.interval_domain import IntervalDomain
from pygeoinf.interval.l2_space import L2Space
from pygeoinf.hilbert_space import EuclideanSpace
from pygeoinf.interval.operators import SOLAOperator
from pygeoinf.interval.functions import Function
from pygeoinf.gaussian_measure import GaussianMeasure
import matplotlib.pyplot as plt
import numpy as np
from pygeoinf.interval.boundary_conditions import BoundaryConditions
from pygeoinf.interval.operators import LaplacianInverseOperator
from pygeoinf.forward_problem import LinearForwardProblem
from pygeoinf.bayesian import LinearBayesianInference
from pygeoinf.linear_solvers import CholeskySolver

## 1. SPACE SETUPS

In [41]:
# Parameters for setting up the spaces
N = 30
N_d = 100
N_p = 10
endpoints = (0, 1)
basis_type = 'sine'
# Create a function domain and spaces
function_domain = IntervalDomain(endpoints[0], endpoints[1])
M = L2Space(N, function_domain, basis_type=basis_type) # model space
D = EuclideanSpace(N_d) # data space
P = EuclideanSpace(N_p) # property space

## 2. MAPPINGS SETUP

In [42]:
# Parameters for setting up the mappings
integration_method_G = 'trapz'
integration_method_T = 'trapz'
n_points_G = 1000
n_points_T = 1000

# Create forward and property mappings
normal_modes_provider = NormalModesProvider(M, gaussian_width_percent_range=(5, 10),
                                            freq_range=(5, 10), random_state=39)
G = SOLAOperator(M, D, normal_modes_provider, integration_method=integration_method_G, n_points=n_points_G)
width = 0.1 # width of the bump target functions
centers = np.linspace(function_domain.a + width / 2, function_domain.b - width / 2, N_p) # centers of the bumps
target_provider = BumpFunctionProvider(M, centers=centers, default_width=width, default_k=0.0)
T = SOLAOperator(M, P, target_provider, integration_method=integration_method_T, n_points=n_points_T)

## 3.TRUE MODEL, FAKE DATA, AND DATA MEASURE SETUP

In [43]:
# Data noise parameters
true_data_noise = 0.1
assumed_data_noise = 0.1

# Create the synthetic true model
m_bar = Function(M, evaluate_callable=lambda x: np.exp(-((x - function_domain.center)/0.5)**2) * np.sin(5 * np.pi * x) + x)

# Generate synthetic observations
d_bar = G(m_bar)  # Clean observations
noise_level = true_data_noise * np.max(d_bar)
np.random.seed(42)  # For reproducibility
d_tilde = d_bar + np.random.normal(0, noise_level, d_bar.shape)  # Noisy observations

# Define data noise covariance
noise_variance = (assumed_data_noise * np.max(d_tilde))**2  # 10% of peak signal
C_D_matrix = noise_variance * np.eye(N_d)

# Create data ERROR measure (zero mean) for LinearForwardProblem
gaussian_D_error = GaussianMeasure.from_covariance_matrix(D, C_D_matrix, expectation=np.zeros(N_d))

# Create data measure (with observed data mean) for visualization
gaussian_D = GaussianMeasure.from_covariance_matrix(D, C_D_matrix, expectation=d_tilde)

## 4.PRIOR MEASURE SETUP

In [44]:
# Prior measure parameters
alpha = 0.1
K = 500  # Number of KL terms (should be ≥ N for full representation)

# Define prior measure parameters
bc_dirichlet = BoundaryConditions(bc_type='dirichlet', left=0, right=0)
  # Correlation length parameter (smaller α → longer correlations)
C_0 = LaplacianInverseOperator(M, bc_dirichlet, alpha=alpha)

# Prior mean (simple linear trend)
m_0 = Function(M, evaluate_callable=lambda x: x)

# Create Gaussian measure on model space
M.create_gaussian_measure(method='kl', kl_expansion=K, covariance=C_0, expectation=m_0)

LaplacianInverseOperator initialized with native solver, dirichlet(left=0, right=0) BCs


## 5.PRIOR PROPERTY ANALYSIS

In [45]:
# Compute property prior by pushing model prior through target operator
prior_P = M.gaussian_measure.affine_mapping(operator=T)
std_P = np.sqrt(np.diag(prior_P.covariance.matrix(dense=True)))

## 6.MODEL POSTERIOR COMPUTATION

In [46]:
# PROBLEM SOLUTION PARAMETERS
solver = CholeskySolver()

# Bayesian update computation
forward_problem = LinearForwardProblem(G, gaussian_D_error)
bayesian_inference = LinearBayesianInference(forward_problem, M.gaussian_measure, T)

# Compute posterior using the built-in solver
posterior_model = bayesian_inference.model_posterior_measure(d_tilde, solver)

# Extract mean and covariance for compatibility
m_tilde = posterior_model.expectation
C_M_matrix = posterior_model.covariance.matrix(dense=True)

# Create new Gaussian measure with sampling capability using dense covariance
mu_M = GaussianMeasure.from_covariance_matrix(M, C_M_matrix, expectation=m_tilde)

# Compute data fit
posterior_prediction = G(m_tilde)
data_misfit = np.linalg.norm(posterior_prediction - d_tilde)

# Show improvement
relative_improvement = 1 - data_misfit / np.linalg.norm(G(m_0) - d_tilde)

## 7.PROPERTY POSTERIOR COMPUTATION

In [47]:
# Compute property posterior using built-in methods

# Use the LinearBayesianInference class to compute property posterior directly
property_posterior = bayesian_inference.property_posterior_measure(d_tilde, solver)

# Extract property mean and covariance
p_tilde = property_posterior.expectation
cov_P_matrix = property_posterior.covariance.matrix(dense=True)

gaussian_P = GaussianMeasure.from_covariance_matrix(P, cov_P_matrix, expectation=p_tilde)
std_P_post = np.sqrt(np.diag(gaussian_P.covariance.matrix(dense=True)))